### Module 10 Assignment 

Lyn Nguyen Nov. 2022

Design a sentiment analysis classifier using the **Sentiment 140** corpus and **NLTK**. Test the classifier using content from Twitter and Reddit. Describe any limitations of your sentiment analyzer. Turn in Python code as a Jupyter for the classifier.


http://help.sentiment140.com/for-students

- data: trainingandtestdata folder 
	
http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/

- how to put together a sentiment analysis classifier

In [268]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


### CLASSIFIER
We get a list of features (words) and their frequencies next. 

In [269]:
# CLASSIFIER
import nltk

def get_words_in_tweets(tweets):  
    """smush all the words in the tweets into a single list"""
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words


def get_word_features(wordlist):
    """ Outputs dictionary, although 
        no frequency count shows up (wordlist)"""
    wordlist = nltk.FreqDist(wordlist)  # FreqDist({'word1': 3, 'word2': 1, etc.}) ordered from most freq to least
    word_features = wordlist.keys()
    return word_features 

# word_features = get_word_features(get_words_in_tweets(tweets))

In [270]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        """word_features is predefined above as list of 
           3+ letter tokens from all tweets combined"""
        features['contains(%s)' % word] = (word in document_words)
    return features

## FINAL PROJECT TRIAL 

In [297]:
# STOP WORDS - from topic_modeling_11
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['.',  ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','year', # remove it if you need punctuation
'#', '://', '/', 'www', '-', 'com', '=', '...', 'org', 'https', '@', '&', "'", '"']) # added for this assignment


# # media stopwords only 
# stop_words = set(['msnbc', 'foxnews', 'npr', 'nytimes', 'cnn', 'usedgov', '@msnbc', '@foxnews', '@npr', '@nytimes', '@cnn', '@usedgov'])
# stop_words


In [300]:
# turn df['tweet'] into token variables 

def tokenize_column(df): 
    '''From hw 8'''
    # input data
    # stem = pd.DataFrame(df)

    # iterate each col's row, use a list to add it back to the dataframe
    tokenized_list = []
    tLenList = []
    msgLen = []
    for ind in df.index: 
        msg = df['text'][ind]           #tweet--> text
        # tokens = word_tokenize(msg)
        tokens = TweetTokenizer().tokenize(msg) # https://stackoverflow.com/questions/34714162/preventing-splitting-at-apostrophies-when-tokenizing-words-using-nltk
        # tknzr = TweetTokenizer()
        # tknzr.tokenize("@Kenichan I haven't dived many times for the ball. Man")

        # tokenized_list.append(tokens) <-- uncomment for non-stopwords token
        # tLenList.append(len(tokens))
        # msgLen.append(len(msg))

        # remove stopwords 
        # print(tokens) # <--- take out later 
        ts = [i.lower() for i in tokens if i.lower() not in stop_words]
        # print(ts)  # <---- take out later 
        tokenized_list.append(ts)
        tLenList.append(len(ts))
        msgLen.append(len(ts))

    df['wordTokenize'] = tokenized_list
    df['tokenLength'] = tLenList
    df['msgLen'] = msgLen

    return df

In [301]:
# testout_token = tokenize_column(fp.head(3)) #<--- RUN ME
# testout_token

# remove stop words 

for row in testout_token.index:
    tokened_text = testout_token['wordTokenize'][row]
    print(tokened_text)
    # ts = [i.lower() for i in tokens if i.lower not in stop_words]
    print("------TAKE OUT STOP WORDS: ----------")
    ts = [i.lower() for i in tokened_text if i.lower() not in stop_words]

    print(ts)

    

['@msnbc', '@maddowblog', 'â', '\x80', '\x9c', 'simpletonâ', '\x80', '\x99', 's', 'defenseâ', '\x80', '\x9d', '?', 'you', 'mean', 'convincing', 'voters', 'that', 'hardworking', 'tax', 'payers', 'shouldnâ', '\x80', '\x99', 't', 'have', 'to', 'pay', 'off', 'someone', 'elseâ', '\x80', '\x99', 's', 'student', 'loan', '?', 'not', 'to', 'mention', ',', 'itâ', '\x80', '\x99', 's', 'illegal', '.']
------TAKE OUT STOP WORDS: ----------
['@msnbc', '@maddowblog', 'â', '\x80', '\x9c', 'simpletonâ', '\x80', '\x99', 'defenseâ', '\x80', '\x9d', 'mean', 'convincing', 'voters', 'hardworking', 'tax', 'payers', 'shouldnâ', '\x80', '\x99', 'pay', 'someone', 'elseâ', '\x80', '\x99', 'student', 'loan', 'mention', 'itâ', '\x80', '\x99', 'illegal']
['@msnbc', '@maddowblog', 'i', 'feel', 'sorry', 'for', 'the', 'suckers', 'who', 'fell', 'for', 'this', 'student', 'loan', 'forgiveness', 'scam', 'and', 'voted', 'for', 'joe', '.']
------TAKE OUT STOP WORDS: ----------
['@msnbc', '@maddowblog', 'feel', 'sorry', 'suc

## Get training data 

In [274]:
input_path = "data/master_annotated_2.csv"
fp = pd.read_csv(input_path, encoding='latin-1')

In [275]:
# add a tokenized list from Twitter text
df1 = tokenize_column(fp)
df1.tail(3)

,Unnamed: 0,experiment_id,experiment_group,text,tweet_id,tweet_likes,retweets,tweet_created_at,user_id,in_reply_to_status_id,...,user_id_char,text_length,text_word_count,opinion_key,opinion_label,opinion_key.1,opinion_label.1,wordTokenize,tokenLength,msgLen
465,465,466,usedgov,@usedgov why are my student loans not transfer...,1.599892e+18,0,0,Mon Dec 05 22:24:29 +0000 2022,7.925171e+17,NaN,...,7.925171e+17,181,33,1,NEUTRAL support,0,Very important,"[@usedgov, student, loans, transferring, nelne...",21,21
466,466,467,foxnews,@FoxNews Just another way of screwing the taxp...,1.599894e+18,0,0,Mon Dec 05 22:32:26 +0000 2022,1.518825e+18,1.599351e+18,...,1.518825e+18,244,45,1,NEUTRAL support,1,Somewhat important,"[@foxnews, another, way, screwing, taxpayers, ...",20,20
467,467,468,foxnews,@FoxNews The Democrats donât seem to be tryi...,1.599904e+18,0,0,Mon Dec 05 23:09:08 +0000 2022,1.586128e+18,1.599901e+18,...,1.586128e+18,194,35,1,NEUTRAL support,1,Somewhat important,"[@foxnews, democrats, donâ, , , seem, trying...",26,26


In [276]:
# check how balanced the data is. total count of negative, neutral, and positive sentiment. 
df1['opinion_label'].value_counts() # sentiment --> opinion_label

NEUTRAL support                     213
FOR student loan forgiveness        123
AGAINST student loan forgiveness    110
cannot judge support                 22
Name: opinion_label, dtype: int64

In [277]:
df1 = df1[['wordTokenize', 'opinion_label']]

In [278]:
df1.tail(2)
# fp_data = fb[['wordTokenize', 'opinion_label']]


,wordTokenize,opinion_label
466,"[@foxnews, another, way, screwing, taxpayers, ...",NEUTRAL support
467,"[@foxnews, democrats, donâ, , , seem, trying...",NEUTRAL support


In [279]:
import random
random.seed(1033)

# https://www.geeksforgeeks.org/how-to-do-train-test-split-using-sklearn-in-python/

from sklearn.model_selection import train_test_split

X = df1.get('wordTokenize')
y = df1.get('opinion_label')

# random_state previously used: 1002
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 102, test_size = .2, shuffle = True)

training_data = pd.concat([X_train, y_train], axis = 1)

In [280]:
def records(df): 
    # https://stackoverflow.com/questions/9758450/pandas-convert-dataframe-to-array-of-tuples
    return df.to_records(index=False).tolist()
training_data = records(training_data)

In [281]:
# input need columns wordTokenize, sentiment 
word_features = get_word_features(get_words_in_tweets(training_data))

In [282]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        """word_features is predefined above as list of 
           3+ letter tokens from all tweets combined"""
        features['contains(%s)' % word] = (word in document_words)
    return features

# apply features to classifier with our feature_extract function 
# it outputs a list of tuple, each tuple holds the "feature dictionary"
training_set = nltk.classify.apply_features(extract_features, training_data)

In [283]:
# train our classifier using our training data set
classifier = nltk.NaiveBayesClassifier.train(training_set)


In [284]:
# test it out 
tweet = "@FoxNews Stop Federal Funding and student loans to attend these schools."

classifier.classify(extract_features(tweet.split()))  # did not take in stopwords 

'NEUTRAL support'

### Apply classifier to Student Loan Twitter Data 

In [285]:
# Test Data 
test_data = pd.concat([X_test, y_test], axis = 1)
test_data.head(4)

,wordTokenize,opinion_label
129,"[@foxnews, hoping, dream, lasts, long, enough,...",NEUTRAL support
441,"[@foxnews, yep, record, high, gas, prices, bor...",NEUTRAL support
59,"[@foxnews, everybody, already, paid, student, ...",AGAINST student loan forgiveness
324,"[@foxnews, like, student, loan, forgiveness, p...",NEUTRAL support


In [286]:
# get index of test data 
test_index = X_test.index
test_index
# grab full dataset using test data index and make it "test_data"
test_data = fp.iloc[test_index]

In [287]:
test_data

,Unnamed: 0,experiment_id,experiment_group,text,tweet_id,tweet_likes,retweets,tweet_created_at,user_id,in_reply_to_status_id,...,user_id_char,text_length,text_word_count,opinion_key,opinion_label,opinion_key.1,opinion_label.1,wordTokenize,tokenLength,msgLen
129,129,130,foxnews,@FoxNews He is hoping his dream lasts long eno...,1.598376e+18,0,0,Thu Dec 01 17:59:36 +0000 2022,1.384685e+18,1.598271e+18,...,1.384685e+18,210,41,1,NEUTRAL support,1,Somewhat important,"[@foxnews, hoping, dream, lasts, long, enough,...",26,26
441,441,442,foxnews,@FoxNews Yep a record in high gas prices borde...,1.599518e+18,0,0,Sun Dec 04 21:36:55 +0000 2022,8.447381e+17,1.599461e+18,...,8.447381e+17,288,42,1,NEUTRAL support,0,Very important,"[@foxnews, yep, record, high, gas, prices, bor...",30,30
59,59,60,foxnews,@FoxNews So everybody that has already paid th...,1.598302e+18,4,0,Thu Dec 01 13:04:54 +0000 2022,1.586865e+18,1.598271e+18,...,1.586865e+18,99,17,2,AGAINST student loan forgiveness,1,Somewhat important,"[@foxnews, everybody, already, paid, student, ...",9,9
324,324,325,foxnews,@FoxNews This is like student loan forgiveness...,1.599111e+18,0,0,Sat Dec 03 18:39:07 +0000 2022,1.549778e+18,1.599109e+18,...,1.549778e+18,118,18,1,NEUTRAL support,0,Very important,"[@foxnews, like, student, loan, forgiveness, p...",14,14
86,86,87,foxnews,@FoxNews Student debt is a national crisis. It...,1.598325e+18,0,0,Thu Dec 01 14:33:58 +0000 2022,1.589460e+18,1.598271e+18,...,1.589460e+18,278,54,1,NEUTRAL support,1,Somewhat important,"[@foxnews, student, debt, national, crisis, ea...",29,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,410,411,foxnews,@FoxNews Looks like we have a new âweâre ...,1.599384e+18,0,0,Sun Dec 04 12:44:52 +0000 2022,1.570620e+18,1.599351e+18,...,1.570620e+18,183,34,0,FOR student loan forgiveness,0,Very important,"[@foxnews, looks, like, new, â, , , weâ, , ...",32,32
331,331,332,foxnews,"@FoxNews Ah yes, the good old switch-a-roo to ...",1.599119e+18,0,0,Sat Dec 03 19:12:15 +0000 2022,1.330691e+18,1.599109e+18,...,1.330691e+18,258,47,0,FOR student loan forgiveness,0,Very important,"[@foxnews, ah, yes, good, old, switch-a-roo, b...",28,28
82,82,83,foxnews,@FoxNews If Biden really wants to help ALL Ame...,1.598320e+18,0,0,Thu Dec 01 14:16:52 +0000 2022,1.586148e+18,1.598271e+18,...,1.586148e+18,157,28,0,FOR student loan forgiveness,0,Very important,"[@foxnews, biden, really, wants, help, america...",19,19
408,408,409,foxnews,@FoxNews If CA wants to do this with their mon...,1.599383e+18,5,0,Sun Dec 04 12:39:55 +0000 2022,1.585820e+18,1.599351e+18,...,1.585820e+18,277,56,1,NEUTRAL support,1,Somewhat important,"[@foxnews, ca, wants, money, rest, country, le...",25,25


In [288]:
# test_data = test_data[['text', 'opinion_label']]
# test_data.head(3)

In [289]:
test_data.head(3)

,Unnamed: 0,experiment_id,experiment_group,text,tweet_id,tweet_likes,retweets,tweet_created_at,user_id,in_reply_to_status_id,...,user_id_char,text_length,text_word_count,opinion_key,opinion_label,opinion_key.1,opinion_label.1,wordTokenize,tokenLength,msgLen
129,129,130,foxnews,@FoxNews He is hoping his dream lasts long eno...,1.598376e+18,0,0,Thu Dec 01 17:59:36 +0000 2022,1.384685e+18,1.598271e+18,...,1.384685e+18,210,41,1,NEUTRAL support,1,Somewhat important,"[@foxnews, hoping, dream, lasts, long, enough,...",26,26
441,441,442,foxnews,@FoxNews Yep a record in high gas prices borde...,1.599518e+18,0,0,Sun Dec 04 21:36:55 +0000 2022,8.447381e+17,1.599461e+18,...,8.447381e+17,288,42,1,NEUTRAL support,0,Very important,"[@foxnews, yep, record, high, gas, prices, bor...",30,30
59,59,60,foxnews,@FoxNews So everybody that has already paid th...,1.598302e+18,4,0,Thu Dec 01 13:04:54 +0000 2022,1.586865e+18,1.598271e+18,...,1.586865e+18,99,17,2,AGAINST student loan forgiveness,1,Somewhat important,"[@foxnews, everybody, already, paid, student, ...",9,9


In [290]:
# RUN TEST DATA THROUGH MODEL 
# tokenize test data first or use pre-tokenized column `wordTokenize` instead of 'text' 
    # test_data = tokenize_column(test_data) # <-- run this if new data isn't yet tokenized. will work if tweet column is named 'text'

class_list = []
for row in test_data.index:
    # msg = test_data['text'][row]
    msg_split = test_data['wordTokenize'][row]
    # msg_split = msg.split()  # doesn't take into account stop words 
    result = classifier.classify(extract_features(msg_split))
    class_list.append(result)
test_data["predicted_sentiment"] = class_list

/var/folders/79/skpws_rj3nd2_9dzb8zqs6900000gn/T/ipykernel_96159/2298502193.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["predicted_sentiment"] = class_list


In [291]:
test_data.head(3)

,Unnamed: 0,experiment_id,experiment_group,text,tweet_id,tweet_likes,retweets,tweet_created_at,user_id,in_reply_to_status_id,...,text_length,text_word_count,opinion_key,opinion_label,opinion_key.1,opinion_label.1,wordTokenize,tokenLength,msgLen,predicted_sentiment
129,129,130,foxnews,@FoxNews He is hoping his dream lasts long eno...,1.598376e+18,0,0,Thu Dec 01 17:59:36 +0000 2022,1.384685e+18,1.598271e+18,...,210,41,1,NEUTRAL support,1,Somewhat important,"[@foxnews, hoping, dream, lasts, long, enough,...",26,26,AGAINST student loan forgiveness
441,441,442,foxnews,@FoxNews Yep a record in high gas prices borde...,1.599518e+18,0,0,Sun Dec 04 21:36:55 +0000 2022,8.447381e+17,1.599461e+18,...,288,42,1,NEUTRAL support,0,Very important,"[@foxnews, yep, record, high, gas, prices, bor...",30,30,NEUTRAL support
59,59,60,foxnews,@FoxNews So everybody that has already paid th...,1.598302e+18,4,0,Thu Dec 01 13:04:54 +0000 2022,1.586865e+18,1.598271e+18,...,99,17,2,AGAINST student loan forgiveness,1,Somewhat important,"[@foxnews, everybody, already, paid, student, ...",9,9,NEUTRAL support


In [292]:
# create column to show if predicted_sentiment is the same as sentiment
conditions = [(test_data['opinion_label']==test_data['predicted_sentiment']),
(test_data['opinion_label'] != test_data['predicted_sentiment'])]
values = ['yes', 'no']
test_data['match'] = np.select(conditions, values)
test_data

/var/folders/79/skpws_rj3nd2_9dzb8zqs6900000gn/T/ipykernel_96159/3574436141.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['match'] = np.select(conditions, values)


,Unnamed: 0,experiment_id,experiment_group,text,tweet_id,tweet_likes,retweets,tweet_created_at,user_id,in_reply_to_status_id,...,text_word_count,opinion_key,opinion_label,opinion_key.1,opinion_label.1,wordTokenize,tokenLength,msgLen,predicted_sentiment,match
129,129,130,foxnews,@FoxNews He is hoping his dream lasts long eno...,1.598376e+18,0,0,Thu Dec 01 17:59:36 +0000 2022,1.384685e+18,1.598271e+18,...,41,1,NEUTRAL support,1,Somewhat important,"[@foxnews, hoping, dream, lasts, long, enough,...",26,26,AGAINST student loan forgiveness,no
441,441,442,foxnews,@FoxNews Yep a record in high gas prices borde...,1.599518e+18,0,0,Sun Dec 04 21:36:55 +0000 2022,8.447381e+17,1.599461e+18,...,42,1,NEUTRAL support,0,Very important,"[@foxnews, yep, record, high, gas, prices, bor...",30,30,NEUTRAL support,yes
59,59,60,foxnews,@FoxNews So everybody that has already paid th...,1.598302e+18,4,0,Thu Dec 01 13:04:54 +0000 2022,1.586865e+18,1.598271e+18,...,17,2,AGAINST student loan forgiveness,1,Somewhat important,"[@foxnews, everybody, already, paid, student, ...",9,9,NEUTRAL support,no
324,324,325,foxnews,@FoxNews This is like student loan forgiveness...,1.599111e+18,0,0,Sat Dec 03 18:39:07 +0000 2022,1.549778e+18,1.599109e+18,...,18,1,NEUTRAL support,0,Very important,"[@foxnews, like, student, loan, forgiveness, p...",14,14,AGAINST student loan forgiveness,no
86,86,87,foxnews,@FoxNews Student debt is a national crisis. It...,1.598325e+18,0,0,Thu Dec 01 14:33:58 +0000 2022,1.589460e+18,1.598271e+18,...,54,1,NEUTRAL support,1,Somewhat important,"[@foxnews, student, debt, national, crisis, ea...",29,29,NEUTRAL support,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,410,411,foxnews,@FoxNews Looks like we have a new âweâre ...,1.599384e+18,0,0,Sun Dec 04 12:44:52 +0000 2022,1.570620e+18,1.599351e+18,...,34,0,FOR student loan forgiveness,0,Very important,"[@foxnews, looks, like, new, â, , , weâ, , ...",32,32,FOR student loan forgiveness,yes
331,331,332,foxnews,"@FoxNews Ah yes, the good old switch-a-roo to ...",1.599119e+18,0,0,Sat Dec 03 19:12:15 +0000 2022,1.330691e+18,1.599109e+18,...,47,0,FOR student loan forgiveness,0,Very important,"[@foxnews, ah, yes, good, old, switch-a-roo, b...",28,28,NEUTRAL support,no
82,82,83,foxnews,@FoxNews If Biden really wants to help ALL Ame...,1.598320e+18,0,0,Thu Dec 01 14:16:52 +0000 2022,1.586148e+18,1.598271e+18,...,28,0,FOR student loan forgiveness,0,Very important,"[@foxnews, biden, really, wants, help, america...",19,19,NEUTRAL support,no
408,408,409,foxnews,@FoxNews If CA wants to do this with their mon...,1.599383e+18,5,0,Sun Dec 04 12:39:55 +0000 2022,1.585820e+18,1.599351e+18,...,56,1,NEUTRAL support,1,Somewhat important,"[@foxnews, ca, wants, money, rest, country, le...",25,25,NEUTRAL support,yes


In [293]:
# count up now many matches
test_data['match'].value_counts() # 48/46 = 61% correct  

no     60
yes    34
Name: match, dtype: int64

How can we calculate **recall** and **precision** of the multiclass model? 

- https://www.analyticsvidhya.com/blog/2021/06/confusion-matrix-for-multi-class-classification/

- https://towardsdatascience.com/is-accuracy-everything-96da9afd540d


In [294]:
from sklearn.metrics import confusion_matrix
# Creating  a confusion matrix,which compares the y_test and y_pred
cm = confusion_matrix(test_data['opinion_label'], test_data['predicted_sentiment'])

In [295]:
# Creating a dataframe for a array-formatted Confusion matrix,so it will be easy for plotting.
cm_df = pd.DataFrame(cm,
                     index = ['FOR student loan forgiveness','NEUTRAL support','AGAINST student loan forgiveness', 'cannot judge support'], 
                     columns = ['FOR student loan forgiveness','NEUTRAL support','AGAINST student loan forgiveness', 'cannot judge support'])

# actual value (row), predicted values (columns)
cm_df

                     

,FOR student loan forgiveness,NEUTRAL support,AGAINST student loan forgiveness,cannot judge support
FOR student loan forgiveness,2,9,10,0
NEUTRAL support,0,2,22,0
AGAINST student loan forgiveness,6,5,30,0
cannot judge support,0,1,7,0


----------------

## EDIT AFTER MODEL COMPLETED

There are 96 matches between `predicted_sentiment` and `sentiment` out of 163 test data points. That is 59% accuracy.

Our model is accurate more than 1/2 of the time. Given its constraints, 59% is acceptable. We believe that if future work address the limitations of this model, the result will improve. Below is a list of the model's limitation: 
- not able to use emoticons 
- not recognizing @username as an entity/subject
- no treatment for commas and periods
- treat lower/upper cases differenlty
- special characters and hashtags are still in test data, unaddressed
- needed to remove stop words from the training model
- A larger training data set might yield better result. We only used 0.125% of the provided Sentiment 140 dataset (2K out of 1.6 million rows). 

Finally, the pre-labeled test data could not be neatly categorize. For example, when we sense "hope" in the text, we would label it as positive, even though there are negative sentiment that prefaces the hope/resolution. 
ex: 
>@POTUS since your student loan forgiveness move is not going to pass muster with the courts, why not do something legitimate and fair. Lock all student loans at 1% interest for all existing and future loans. #StudentLoans2022 #loanforgiveness #studentloans #college

The manual we gave this tweet was 'positive' but our model categorizes it as 'negative'. 

In [296]:
contain_values = test_data[test_data['tweet'].str.contains('@POTUS since your student loan forgiveness move is not going to pass muster with the courts')]
contain_values

KeyError: 'tweet'

-------------
